In [1]:
import os
import time
import math
import itertools
import yaml
import numpy as np
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from scipy.optimize import minimize, Bounds
from hypernet_MLP import Hypernet_MLP
from hypernet_trans import Hypernet_trans
from functions_hv_python3 import HyperVolume

In [2]:
def set_seed(seed=702):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed(702)

In [3]:
case = case = "_Ex_7_2"

# Const

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


In [5]:
GROUND_TRUTH_FILE = f"../4_Pareto_front/test/{case}/pf_dynamic_true.npy" 

if os.path.exists(GROUND_TRUTH_FILE):
    pf_true = np.load(GROUND_TRUTH_FILE)
    print(f"✅ Đã tải ground truth Pareto front từ: {GROUND_TRUTH_FILE}, Shape: {pf_true.shape}")
else:
    print(f"❌ KHÔNG tìm thấy file ground truth tại: {GROUND_TRUTH_FILE}")

✅ Đã tải ground truth Pareto front từ: ../4_Pareto_front/test/_Ex_7_2/pf_dynamic_true.npy, Shape: (20, 2)


# Def

In [6]:
import autograd.numpy as np
from scipy.optimize import minimize            
        
class Projection:
    def __init__(self, cons, bounds, dim, proj_type='euclid'):
        self.cons = cons
        self.bounds = bounds
        self.dim = dim
        self.proj_type = proj_type
        
        if self.proj_type == 'qplus':
            self.objective_func = self._obj_positive_diff
        elif self.proj_type == 'euclid':
            self.objective_func = self._obj_l2_norm
        else:
            print(f"Phép chiếu {self.objective_func} không cài đặt, chọn 'qplus' hoặc 'euclid'")

    def _obj_l2_norm(self, x, y):
        return np.sqrt(np.sum((x - y)**2))
    
    def _obj_positive_diff(self, x, y):
        v = np.maximum(y - x, 0) 
        return np.sum(v**2)

    def project(self, target_point):
        init_point = np.random.rand(1, self.dim).tolist()[0]
        
        res = minimize(
            self.objective_func,
            init_point,
            args=(target_point, ),
            constraints=self.cons,
            bounds=self.bounds,
            options={'disp': False}
        )
        
        optim_point = res.x
        
        if self.proj_type == 'qplus':
            return target_point - np.maximum(target_point - optim_point, 0)
        else:
            return optim_point

class Problem():
    def __init__(self, f, dim_x, dim_y, proj_C, proj_Qplus):
        self.f = f
        self.dim_x = dim_x
        self.dim_y = dim_y
        self.proj_C = proj_C
        self.proj_Qplus = proj_Qplus
    
    def objective_func(self, x):
        vals = [func(x) for func in self.f]
        return np.concatenate(vals)  

In [7]:
def f1(x):    return (x[0]**2 + x[1]**2)/50
def f2(x):    return ((x[0] - 5)**2 + (x[1] - 5)**2)/50
def f(x):    return np.array([
    (x[0]**2 + x[1]**2)/50,
    ((x[0] - 5)**2 + (x[1] - 5)**2)/50])
#--------------- C --------------------#
bounds_x = Bounds([0,0],[5, 5])

#--------------- Q --------------------#
def q1(y):    return 0.2**2 - (y[0] - 0.4)**2 - (y[1] - 0.4)**2

def q_plus(y):
    center = 0.4
    radius_sq = 0.2**2  
    dx = np.maximum(0, y[0] - center)
    dy = np.maximum(0, y[1] - center)
    return radius_sq - (dx**2 + dy**2)
# Hàm dùng cho Projection 
cons_C = ()
dim_x = 2
cons_Q = ({'type': 'ineq', 'fun' : q1,},)
cons_Qplus = ({'type': 'ineq', 'fun': q_plus},)
dim_y = 2
# Setup Projections
proj_C_handler = Projection(cons=cons_C, bounds=bounds_x, dim=dim_x, proj_type='euclid')
proj_Q_handler = Projection(cons=cons_Q, bounds=None, dim=dim_y, proj_type='qplus')
# Setup Problem
prob = Problem(
    f=[f1, f2], 
    dim_x=dim_x, dim_y=dim_y,
    proj_C=proj_C_handler.project,
    proj_Qplus=proj_Q_handler.project
)
z_star = np.array([0.0, 0.0])
x_init = np.array([-10.0, -10.0])

In [9]:
def evaluate_objectives_single(functions, x_tensor):
    vals = []
    for func in functions:
        val = func(x_tensor)
        if not torch.is_tensor(val): val = torch.tensor(val, dtype=torch.float32, device=x_tensor.device)
        vals.append(val)
    return torch.stack(vals).reshape(-1)

def calculate_mse_igd(pf_pred, pf_true):
    if len(pf_pred) == 0: return np.inf
    total_dist_sq = 0
    # Với mỗi điểm ground truth, tìm điểm dự đoán gần nhất
    for p_true in pf_true:
        dists_sq = np.sum((pf_pred - p_true)**2, axis=1)
        total_dist_sq += np.min(dists_sq)
    return total_dist_sq / len(pf_true)

def calculate_mse(pf_pred, pf_true):
    pf_pred_ = np.array(pf_pred)
    pf_true_ = np.array(pf_true)
    
    if pf_pred_.shape != pf_true_.shape:
        print(f"⚠️ Warning: Shape mismatch {pf_pred_.shape} vs {pf_true_.shape}. MSE có thể không chính xác.")
        return np.inf

    return np.mean((pf_pred_ - pf_true_)**2)
def calculate_hv_score(pareto_f, prob, ref_point):
    """Tính Hypervolume, loại bỏ điểm vi phạm ràng buộc Q+"""
    valid_points = []
    tol = 1e-3
    for point in pareto_f:
        # Kiểm tra feasibility với Q+
        point_proj = prob.proj_Qplus(point)
        dist_Q = np.linalg.norm(point - point_proj)
        
        # Kiểm tra nằm trong vùng Reference
        is_dominated_by_ref = np.all(point < ref_point)
        
        if dist_Q < tol and is_dominated_by_ref:
            valid_points.append(point.tolist())
            
    if len(valid_points) < 2: return 0.0
    
    hv = HyperVolume(ref_point)
    return hv.compute(valid_points)

In [10]:
def evaluate_objectives_single(functions, x_tensor):
    """
    Hàm phụ trợ: Tính giá trị f1(x), f2(x) cho 1 mẫu x duy nhất.
    """
    vals = []
    for func in functions:
        val = func(x_tensor)
        if not torch.is_tensor(val):
            val = torch.tensor(val, dtype=torch.float32)
        vals.append(val)
    return torch.stack(vals).reshape(-1)

def train_hypernet(hypernet, prob, z_star, 
                   num_epochs=1000, 
                   lr=1e-3, 
                   num_partitions=100, 
                   lr_step_size=300, 
                   lr_gamma=0.5,
                   # --- THAM SỐ THUẬT TOÁN 2-A (Monotonic Penalty) ---
                   # Cả hai đều TĂNG dần để đảm bảo Feasibility
                   beta_C_0=1.0,    # Giá trị khởi tạo cho C
                   beta_C_max=1000.0, # Giá trị tối đa cho C
                   rho_C=1.01,      # Tỷ lệ tăng cho C (VD: 1.01 = +1%/step)
                   
                   beta_Q_0=1.0,    # Giá trị khởi tạo cho Q (Algorithm 2-A: Tăng Q)
                   beta_Q_max=1000.0, # Giá trị tối đa cho Q
                   rho_Q=1.01,      # Tỷ lệ tăng cho Q
                   verbose=True): 
    
    # 1. Khởi tạo
    optimizer = optim.Adam(hypernet.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_step_size, gamma=lr_gamma)
    
    # Đảm bảo z_star chuẩn shape
    z_star_tensor = torch.tensor(z_star, dtype=torch.float32).view(1, -1)
    
    # Khởi tạo hệ số phạt (Step 1 của Alg 2-A)
    beta_C = beta_C_0
    beta_Q = beta_Q_0
    
    angle_step = (math.pi / 2) / num_partitions
    
    if verbose:
        print(f"=== TRAIN HYPERNET (Algorithm 2-A: Monotonic Penalty) ===")
        print(f"Constraint C: Start {beta_C_0} -> Max {beta_C_max} (Rate {rho_C})")
        print(f"Constraint Q: Start {beta_Q_0} -> Max {beta_Q_max} (Rate {rho_Q})")
    
    for epoch in range(num_epochs):
        hypernet.train()
        optimizer.zero_grad()
        
        # --------------------------------------------------------
        # 1. Lấy mẫu phân tầng (Stratified Sampling - Step 4,5 of Alg 2-A)
        # --------------------------------------------------------
        starts = torch.arange(num_partitions) * angle_step
        noise = torch.rand(num_partitions) * angle_step
        thetas = starts + noise 
        
        r_batch_np = np.stack([np.cos(thetas.numpy()), np.sin(thetas.numpy())], axis=1)
        r_tensor_batch = torch.tensor(r_batch_np, dtype=torch.float32)
        
        # --------------------------------------------------------
        # 2. Lan truyền xuôi & Tính Loss (Step 8-18 of Alg 2-A)
        # --------------------------------------------------------
        
        # Forward pass (Sequential để tránh lỗi shape của Hypernet)
        x_pred_list = []
        for i in range(num_partitions):
            r_single = r_tensor_batch[i].unsqueeze(0)
            x_single = hypernet(r_single)
            x_pred_list.append(x_single)
            
        x_vec_batch = torch.cat(x_pred_list, dim=0) # Batch output
        
        # Loop tính Loss thành phần (do Scipy projection không hỗ trợ batch)
        x_np_batch = x_vec_batch.detach().cpu().numpy()
        loss_C_list = []
        loss_Q_list = []
        y_pred_list = []
        
        for i in range(num_partitions):
            x_i_tensor = x_vec_batch[i].reshape(-1) 
            x_i_np = x_i_tensor.detach().cpu().numpy()
            
            # a. Loss C: ||x - P_C(x)||^2
            x_proj_i_np = prob.proj_C(x_i_np)
            x_proj_i_tensor = torch.tensor(x_proj_i_np, dtype=torch.float32)
            loss_C_list.append(torch.sum((x_i_tensor - x_proj_i_tensor)**2))
            
            # b. F(x)
            y_pred_i = evaluate_objectives_single(prob.f, x_i_tensor)
            y_pred_list.append(y_pred_i)
            
            # c. Loss Q: ||y - P_Q+(y)||^2
            y_i_np = y_pred_i.detach().cpu().numpy()
            y_proj_i_np = prob.proj_Qplus(y_i_np)
            y_proj_i_tensor = torch.tensor(y_proj_i_np, dtype=torch.float32)
            loss_Q_list.append(torch.sum((y_pred_i - y_proj_i_tensor)**2))
        
        # Tính trung bình Loss (Mean L_C, Mean L_Q)
        y_pred_batch = torch.stack(y_pred_list)
        L_bar_C = torch.mean(torch.stack(loss_C_list))
        L_bar_Q = torch.mean(torch.stack(loss_Q_list))
        
        # --------------------------------------------------------
        # 3. Tính Loss Mục tiêu Chebyshev (Step 20-21 of Alg 2-A)
        # --------------------------------------------------------
        diff = y_pred_batch - z_star_tensor
        weighted_diff = r_tensor_batch * diff
        max_vals, _ = torch.max(weighted_diff, dim=1)
        L_Obj = torch.mean(max_vals)
        
        # --------------------------------------------------------
        # 4. Tổng hợp và Cập nhật (Step 24-27 of Alg 2-A)
        # --------------------------------------------------------
        # L_Total = L_Obj + β_C * L_C + β_Q * L_Q
        total_loss = L_Obj + (beta_C * L_bar_C) + (beta_Q * L_bar_Q)
        
        total_loss.backward()
        optimizer.step()
        scheduler.step()
        
        # --------------------------------------------------------
        # 5. Cập nhật hệ số Phạt (Step 30-31 of Alg 2-A)
        # --------------------------------------------------------
        # Cả hai hệ số đều TĂNG dần
        beta_C = min(beta_C_max, beta_C * rho_C)
        beta_Q = min(beta_Q_max, beta_Q * rho_Q)
        
        # Logging
        if verbose and epoch % 100 == 0:
            current_lr = scheduler.get_last_lr()[0]
            print(f"Epoch {epoch}: Total={total_loss.item():.3f} "
                  f"(Obj={L_Obj.item():.4f}, C={L_bar_C.item():.5f}, Q={L_bar_Q.item():.5f}) "
                  f"|| BetaC={beta_C:.1f}, BetaQ={beta_Q:.1f}")
            
    return hypernet

In [11]:
def evaluate_model(hypernet, prob, test_rays, pf_true, metric_func):
    hypernet.eval()
    pf_pred = []
    rays_tensor = torch.tensor(test_rays, dtype=torch.float32, device=device)
    
    with torch.no_grad():
        # Sequential infer to avoid shape issues
        for i in range(len(rays_tensor)):
            r_single = rays_tensor[i].unsqueeze(0)
            x_raw = hypernet(r_single).squeeze().cpu().numpy()
            
            # Project to C for fair comparison
            x_proj = prob.proj_C(x_raw)
            val = [func(x_proj) for func in prob.f]
            pf_pred.append(val)
            
    pf_pred = np.array(pf_pred)
    score = metric_func(pf_pred, pf_true)
    return score, pf_pred
def evaluate_model_hv(hypernet, prob, test_rays, ref_point):
    """
    Đánh giá model dựa trên chỉ số Hypervolume (HV).
    """
    hypernet.eval()
    pf_pred = []
    
    # Chuyển test rays lên device
    rays_tensor = torch.tensor(test_rays, dtype=torch.float32, device=device)
    
    with torch.no_grad():
        # Sequential infer để tránh lỗi shape của model
        for i in range(len(rays_tensor)):
            r_single = rays_tensor[i].unsqueeze(0)
            
            # Forward pass
            x_raw = hypernet(r_single).squeeze().cpu().numpy()
            
            # Chiếu lên C (Bắt buộc để đảm bảo feasibility đầu vào)
            x_proj = prob.proj_C(x_raw)
            
            # Tính giá trị hàm mục tiêu
            val = [func(x_proj) for func in prob.f]
            pf_pred.append(val)
            
    pf_pred = np.array(pf_pred)
    
    # Tính HV Score (Sử dụng hàm đã có từ bước trước)
    # Lưu ý: Hàm này tự lọc bỏ các điểm vi phạm Q+
    score = calculate_hv_score(pf_pred, prob, ref_point)
    
    return score, pf_pred

# Config

In [16]:
param_grid = {
    'lr': [1e-3],
    'num_epochs': [500],
    
    'num_partitions': [20],
    
    # Tham số thuật toán 2-A: Tăng dần penalty
    'beta_C_0': [1.0],
    'beta_Q_0': [1.0],
    'rho_C': [1.01, 1.05], 
    'rho_Q': [1.01, 1.05],
    
    # Cố định Max để tránh grid quá lớn 
    'beta_C_max': [1000.0],
    'beta_Q_max': [1000.0]
}

config_path='../4_Pareto_front/config.yaml'
with open(config_path, 'r') as f:
    cfg = yaml.safe_load(f)
test_rays = np.array(cfg['data']['test_ray'])
ref_point = np.array([1.3927, 1.3927])

# Run

In [17]:
models = ["trans", "MLP"]
mode_tests = ["HV"] 

In [18]:
results = []
best_scores_tracker = {}
save_dir = f"model/{case}"

In [19]:
for model_name in models:
    print(f"\n🔹 Đang train Model: {model_name}")
    
    # Với HV, ta tìm MAX (ban đầu set là -1 hoặc 0)
    current_best_score = -1.0 
    current_best_config = None
    
    # Tạo lưới tham số
    keys, values = zip(*param_grid.items())
    param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
    
    for idx, params in enumerate(param_combinations):
        print(f"\n   >>> Config {idx+1}/{len(param_combinations)}")
        
        # 1. Init Model
        if model_name == "MLP":
            model = Hypernet_MLP(ray_hidden_dim=32, out_dim=dim_x, n_tasks=2)
        else:
            model = Hypernet_trans(ray_hidden_dim=32, out_dim=dim_x, n_tasks=2)
        
        # 2. Train (Algorithm 2-A: Monotonic Penalty)
        start_time = time.time()
        # Lưu ý: Tắt verbose=True để log gọn hơn nếu chạy nhiều config
        trained_model = train_hypernet(
            model, prob, z_star, 
            num_epochs=params['num_epochs'],
            lr=params['lr'],
            num_partitions=params['num_partitions'], 
            # Param phạt động
            beta_C_0=params['beta_C_0'],
            beta_C_max=params['beta_C_max'],
            rho_C=params['rho_C'],
            beta_Q_0=params['beta_Q_0'],
            beta_Q_max=params['beta_Q_max'],
            rho_Q=params['rho_Q'],
            verbose=False 
        )
        train_time = time.time() - start_time

        # 3. Evaluate theo HV
        # Không cần pf_true (ground truth) vì HV là metric unsupervised
        score, pf_pred = evaluate_model_hv(trained_model, prob, test_rays, ref_point)

        print(f"      ⏱️ Time: {train_time:.2f}s | 📈 HV Score: {score:.6f}")

        # 4. Save result
        res = {
            'model_type': model_name,
            'metric_type': "HV",
            'config_id': idx,
            'params': params,
            'score': score,
            'time': train_time
        }
        results.append(res)

        # 5. Update Best (MAXIMIZATION)
        if score > current_best_score:
            current_best_score = score
            current_best_config = params
            
            save_path = f"{save_dir}/best_{model_name}_HV.pth"
            torch.save(trained_model.state_dict(), save_path)
            print(f"      🏆 New Best Found! Saved to: {save_path}")

    print(f"\n✅ Hoàn thành {model_name}. Best HV: {current_best_score:.6f}")
    print(f"   Best Config: {current_best_config}")


🔹 Đang train Model: trans

   >>> Config 1/4


C:\Users\Admin\AppData\Local\Temp\ipykernel_7036\519494600.py:28: DeprecationWarning: scipy.optimize: The `disp` and `iprint` options of the L-BFGS-B solver are deprecated and will be removed in SciPy 1.18.0.
  res = minimize(


      ⏱️ Time: 65.32s | 📈 HV Score: 1.675372
      🏆 New Best Found! Saved to: model/_Ex_7_2/best_trans_HV.pth

   >>> Config 2/4
      ⏱️ Time: 64.38s | 📈 HV Score: 1.677715
      🏆 New Best Found! Saved to: model/_Ex_7_2/best_trans_HV.pth

   >>> Config 3/4
      ⏱️ Time: 65.25s | 📈 HV Score: 1.675509

   >>> Config 4/4
      ⏱️ Time: 65.22s | 📈 HV Score: 1.687903
      🏆 New Best Found! Saved to: model/_Ex_7_2/best_trans_HV.pth

✅ Hoàn thành trans. Best HV: 1.687903
   Best Config: {'lr': 0.001, 'num_epochs': 500, 'num_partitions': 20, 'beta_C_0': 1.0, 'beta_Q_0': 1.0, 'rho_C': 1.05, 'rho_Q': 1.05, 'beta_C_max': 1000.0, 'beta_Q_max': 1000.0}

🔹 Đang train Model: MLP

   >>> Config 1/4
      ⏱️ Time: 58.87s | 📈 HV Score: 1.689825
      🏆 New Best Found! Saved to: model/_Ex_7_2/best_MLP_HV.pth

   >>> Config 2/4
      ⏱️ Time: 59.63s | 📈 HV Score: 1.683531

   >>> Config 3/4
      ⏱️ Time: 59.62s | 📈 HV Score: 1.679540

   >>> Config 4/4
      ⏱️ Time: 59.61s | 📈 HV Score: 1.684209

✅

In [22]:
import pandas as pd
df_results = pd.DataFrame(results)
print("\n=== TỔNG HỢP KẾT QUẢ ===")
df_results.sort_values(by=['score'], ascending=False)


=== TỔNG HỢP KẾT QUẢ ===


,model_type,metric_type,config_id,params,score,time
4,MLP,HV,0,"{'lr': 0.001, 'num_epochs': 500, 'num_partitio...",1.689825,58.870755
3,trans,HV,3,"{'lr': 0.001, 'num_epochs': 500, 'num_partitio...",1.687903,65.224389
7,MLP,HV,3,"{'lr': 0.001, 'num_epochs': 500, 'num_partitio...",1.684209,59.613948
5,MLP,HV,1,"{'lr': 0.001, 'num_epochs': 500, 'num_partitio...",1.683531,59.629263
6,MLP,HV,2,"{'lr': 0.001, 'num_epochs': 500, 'num_partitio...",1.679540,59.623288
1,trans,HV,1,"{'lr': 0.001, 'num_epochs': 500, 'num_partitio...",1.677715,64.376963
2,trans,HV,2,"{'lr': 0.001, 'num_epochs': 500, 'num_partitio...",1.675509,65.247506
0,trans,HV,0,"{'lr': 0.001, 'num_epochs': 500, 'num_partitio...",1.675372,65.324842


# Viz

In [ ]:
print("\n" + "="*40)
print(f"🏆 BEST CONFIGURATION FOUND (IGD={best_igd:.4f})")
print("="*40)
for k, v in best_config.items():
    print(f"{k}: {v}")

# Sắp xếp kết quả theo IGD
sorted_results = sorted(results, key=lambda x: x['igd'])
print("\nTop 5 Configs:")
for i in range(min(5, len(sorted_results))):
    r = sorted_results[i]
    print(f"Rank {i+1}: IGD={r['igd']:.4f} | Params={r['params']}")

# --- Plotting ---
plt.figure(figsize=(10, 6))

# Plot Ground Truth
if pf_true is not None:
    plt.scatter(pf_true[:, 0], pf_true[:, 1], c='gray', alpha=0.5, label='Ground Truth', s=20)

# Plot Best Prediction
if best_pf_pred is not None:
    plt.scatter(best_pf_pred[:, 0], best_pf_pred[:, 1], c='red', marker='x', label='Best Prediction', s=40)

# Plot Ideal Point
plt.scatter(z_star[0], z_star[1], c='green', marker='*', s=150, label='Ideal Point (z*)')

plt.xlabel('f1')
plt.ylabel('f2')
plt.title(f'Pareto Front Approximation (Best IGD: {best_igd:.4f})')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
shutdown -s -t 10800
